In [ ]:
import os, sys 
#to be able to interact with Google Drive's operating system
from google.colab import drive 
#drive is a module that allows us use Python to interact with google drive
drive.mount('/content/gdrive') 
#mounting google drive allows us to work with its contents
nb_path = '/content/notebooks'
os.symlink('/content/gdrive/My Drive/Colab Notebooks', nb_path)
sys.path.insert(0, nb_path)  # or append(nb_path)
#The last three lines are what changes the path of the file.c

Mounted at /content/gdrive


Install the required package

In [ ]:
pip install dcor

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install tigramite==4.2.2.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 420 kB 4.3 MB/s 


Import requied libraries

In [ ]:
# Imports
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline     
## use `%matplotlib notebook` for interactive figures
# plt.style.use('ggplot')
#import sklearn
import tigramite
from tigramite import data_processing as pp
from tigramite import plotting as tp
from tigramite.pcmci import PCMCI
from tigramite.independence_tests import ParCorr, GPDC, CMIknn, CMIsymb

/content/notebooks/tigramite/independence_tests/gpdc_torch.py:18: UserWarning: No module named 'gpytorch'
  warnings.warn(str(e))


Upload the dataset we are interested to find the lagged causal links

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving x4_6.csv to x4_6.csv


In [ ]:
import io
import pandas as pd
df = pd.read_csv(io.BytesIO(uploaded['x4_6.csv']))# Dataset is now stored in a Pandas Dataframe

In [ ]:
df=df.to_numpy()
print(df)

[[0.89000477 1.5590901  2.16003024 2.73205091]
 [1.46768948 1.63955773 0.89680312 2.29657245]
 [1.0191178  2.02218761 1.07532475 2.92723811]
 ...
 [1.47998472 1.96889639 1.69444006 1.99635952]
 [1.60823083 2.33349668 1.80267457 2.41773215]
 [1.79460661 2.00964625 2.3317092  3.50756428]]


In [ ]:
# Initialize dataframe object, specify time axis and variable names
var_names = [r'$X^1$', r'$X^2$', r'$X^3$', r'$X^4$']
dataframe = pp.DataFrame(df, 
                         datatime = np.arange(len(df)), 
                         var_names=var_names)

Perform the test using the parameters: significance level of the CI tests, minimum and maximum period of lags. Record the lagged links for the further steps.

PCMCI+ for full graph

In [ ]:
cond_ind_test = CMIknn()
pcmci = PCMCI(dataframe=dataframe, cond_ind_test=cond_ind_test)
results = pcmci.run_pcmciplus(tau_min=0, tau_max=6, pc_alpha=0.05)
pcmci.print_results(results, alpha_level=0.05)


## Significant links at alpha = 0.05:

    Variable $X^1$ has 1 link(s):
        ($X^1$ -1): pval = 0.00000 | val =  0.059

    Variable $X^2$ has 3 link(s):
        ($X^1$ -1): pval = 0.00000 | val =  0.019
        ($X^2$ -1): pval = 0.00000 | val =  0.017
        ($X^2$ -6): pval = 0.04600 | val =  0.013

    Variable $X^3$ has 1 link(s):
        ($X^2$ -6): pval = 0.00000 | val =  0.028

    Variable $X^4$ has 2 link(s):
        ($X^3$ -3): pval = 0.00000 | val =  0.063
        ($X^3$  0): pval = 0.00000 | val =  0.045

## Ambiguous triples:

    ($X^3$ -3) --> $X^4$ o-o $X^3$


33 mins

PCMCI+ for lagged adjacency

In [ ]:
cond_ind_test = CMIknn()
pcmci = PCMCI(dataframe=dataframe, cond_ind_test=cond_ind_test)
results = pcmci.run_pcmciplus(tau_min=1, tau_max=6, pc_alpha=0.05)
pcmci.print_results(results, alpha_level=0.05)


## Significant links at alpha = 0.05:

    Variable $X^1$ has 1 link(s):
        ($X^1$ -1): pval = 0.00000 | val =  0.059

    Variable $X^2$ has 2 link(s):
        ($X^1$ -1): pval = 0.00000 | val =  0.019
        ($X^2$ -1): pval = 0.00100 | val =  0.017

    Variable $X^3$ has 1 link(s):
        ($X^2$ -6): pval = 0.00000 | val =  0.032

    Variable $X^4$ has 2 link(s):
        ($X^3$ -3): pval = 0.00000 | val =  0.064
        ($X^2$ -6): pval = 0.00400 | val =  0.016


28 mins